In [1]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import nbimporter
import scipy.io as sio
import pandas as pd

%run config.ipynb

In [2]:
X=sio.loadmat(hsidir)["pavia"]
Y=sio.loadmat(hsilabel)['pavia_gt']

X_data=[]
Y_data=[]

indices={}
for i in range(9):
    indices[i]=[]

r=3
c=3

freq=np.zeros((9,), dtype=np.int32)
i=0

while True:

    if c+7>=X.shape[1]:
        c=3
        r+=1

    if r>X.shape[0]-4:
        break

    if Y[r][c]!=0:
        
        X_data.append(X[r-3:r+4, c-3:c+4, :])
        Y_data.append(Y[r][c]-1)
        indices[Y[r][c]-1].append(i)
        i+=1

    c+=1

In [3]:
X_train=[]
Y_train=[]

X_test=[]
Y_test=[]
for i in range(9):
    
    ind=np.random.choice(range(len(indices[i])), size=min(140, len(indices[i])), replace=False)
    ind.sort()
    k=0
    
    for j in range(len(indices[i])):
        if k<140 and ind[k]==j:
            X_train.append(X_data[indices[i][j]])
            Y_train.append(Y_data[indices[i][j]])
            k+=1
        else:
            X_test.append(X_data[indices[i][j]])
            Y_test.append(Y_data[indices[i][j]])
            
for i in range(len(X_train)):
    X_train[i]=np.reshape(X_train[i], (7,7,102,1))
    
for i in range(len(X_test)):
    X_test[i]=np.reshape(X_test[i], (7,7,102,1))
    

In [4]:
class Data(Dataset):
    
    def __init__(self, X, Y):
        
        self.X=X
        self.Y=Y        

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        
        inp=self.X[idx]
        out=np.zeros((9,))
        out[self.Y[idx]]=1
        
        inp=torch.from_numpy(inp.astype(np.float32)).float()
        out=torch.from_numpy(out).long()
        
        return inp, out

In [5]:
batch_size=32
dataset=Data(X_train, Y_train)
lo=DataLoader(dataset,batch_size=batch_size,num_workers=0,shuffle=True)